In [3]:
from selenium.webdriver.support.ui import Select
import time
import googlemaps
import pandas as pd
import tqdm
from selenium import webdriver
import json


CRAWLING_SIZE = 100
CENTER = "서울특별시 강남구 역삼동 858 강남역"

# googlemaps 키 읽어오기
secret_file = 'secrets.json'
with open(secret_file) as f:
    secrets = json.loads(f.read())

    def get_secret(setting, secrets=secrets):
        try:
            return secrets[setting]
        except KeyError:
            error_msg = "Set the {} environment variable".format(setting)
            raise error_msg

    GOOGLEMAPS_KEY = get_secret("GOOGLEMAPS_KEY")

    gmaps = googlemaps.Client(key=GOOGLEMAPS_KEY)

# 셀레늄 시작
driver = webdriver.Chrome('C:/Temp/chromedriver')
driver.implicitly_wait(3) 

driver.get('https://www.yogiyo.co.kr/mobile/#/')

# 검색창에 주소 입력
search = driver.find_element_by_css_selector("#search > div > form > input")
search.clear()
search.send_keys(CENTER)
driver.find_element_by_css_selector("#button_search_address > button.btn.btn-default.ico-pick").click()

# 리뷰순으로 정렬
time.sleep(5)
sorting = Select(driver.find_element_by_css_selector("#content > div > div.row.restaurant-list-info > div.list-option > div > select"))
sorting.select_by_value("review_count")

restaurant_sort_review = pd.DataFrame(columns=["name", "lat", "lng"])

# 220509 기준
# 100%|██████████| 100/100 [14:23<00:00,  8.64s/it]
for i in tqdm.trange(1, CRAWLING_SIZE+1):
    # 페이지 아래로 옮기기
    # 1~60: 0
    # 61~120: 1
    for j in range((i+1) // 60):
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    
    # 음식점 클릭
    time.sleep(3)
    driver.implicitly_wait(3)
    tmp2 = driver.find_element_by_css_selector(f"#content > div > div:nth-child(5) > div > div > div:nth-child({i}) > div").click()
    # 음식점 이름
    time.sleep(2)
    driver.find_element_by_css_selector("#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(3) > a")
    # 음식점 정보 클릭
    time.sleep(1)
    driver.implicitly_wait(3)
    driver.find_element_by_css_selector('#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > ul > li:nth-child(3)').click()
    # 음식점 주소
    time.sleep(1)
    name = driver.find_element_by_css_selector("#content > div.restaurant-detail.row.ng-scope > div.col-sm-8 > div.restaurant-info > div.restaurant-title > span").text
    address = driver.find_element_by_xpath('//*[@ng-show="restaurant.address.length > 0"]/span').text
    # 주소를 위도 경도로 변경
    geo_location = gmaps.geocode(address)[0].get('geometry')
    lat, lng = geo_location['location'].values()
    restaurant_sort_review.loc[i] = [name, lat, lng]
    
    # 음식점리스트로 돌아가기
    driver.back()

  3%|▎         | 3/100 [00:27<14:42,  9.09s/it]


KeyboardInterrupt: 

In [20]:
display(restaurant_sort_review)

restaurant_sort_review.to_csv("강남역_배달음식점_리뷰순")

,name,lat,lng
1,천미향1호점,37.516816,127.034269
2,굽네치킨&피자-역삼2동점,37.493376,127.042338
3,현경(본점),37.516299,127.039363
4,스쿨푸드-논현점,37.508626,127.036354
5,교촌치킨-역삼1호점,37.493704,127.035359
...,...,...,...
96,노랑통닭-역삼점,37.502126,127.035754
97,뉴욕버거앤치킨-신논현점,37.506567,127.022607
98,요리잘하는닭,37.509079,127.022955
99,BHC-한티역점,37.498161,127.052835
